In [1]:
!pip install yfinance

In [18]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt


Question 1: IPO Filings Web Scraping and Data Processing

What's the total sum ($m) of 2023 filings that happened on Fridays?

Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/ Convert the 'Filing Date' to datetime(), 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs). Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given). You may be inspired by the function extract_numbers() in [Code Snippet 4], or you can write your own function to "parse" a string. Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)

Find the total sum in $m (millions of USD, closest INTEGER number) for all filings during 2023, which happened on Fridays (Date.dt.dayofweek()==4). You should see 32 records in total, 25 of it is not null.

(additional: you can read about S-1 IPO filing to understand the context)

In [11]:
import pandas as pd
import requests

# Function to extract numbers from a string
def extract_numbers(text):
    numbers = re.findall(r'\d+\.?\d*', text)
    if len(numbers) == 1:
        return float(numbers[0])
    elif len(numbers) == 2:
        return (float(numbers[0]) + float(numbers[1])) / 2
    else:
        return None

# Fetching data from the endpoint
url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)
ipo = pd.read_html(response.text)
ipo = ipo_dfs[0]

# Convert 'Filing Date' to datetime
ipo['Filing Date'] = pd.to_datetime(ipo['Filing Date'])

# Convert 'Shares Offered' to float64
ipo['Shares Offered'] = ipo['Shares Offered'].replace('-', float('nan')).astype(float)

# Define 'Avg_price' based on 'Price Range'
ipo['Avg_price'] = ipo['Price Range'].apply(extract_numbers)

# Define 'Shares_offered_value'
ipo['Shares_offered_value'] = ipo['Shares Offered'] * ipo['Avg_price']

# Filter data for filings that occurred on Fridays during 2023
friday_filings_2023 = ipo[(ipo['Filing Date'].dt.year == 2023) & (ipo['Filing Date'].dt.dayofweek == 4)]

# Calculate total sum in $m
total_sum_millions_usd = int(round(friday_filings_2023['Shares_offered_value'].sum() / 1e6))

print("Total sum in $m for filings on Fridays in 2023:", total_sum_millions_usd)

Total sum in $m for filings on Fridays in 2023: 286


Question 2: IPOs "Fixed days hold" strategy

Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?

Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/). Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance.

Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks. Some of the tickers like 'DYCQ' and 'LEGT' were on the market less than 30 days (11 and 21 days, respectively). Let's leave them in the dataset; it just means that you couldn't hold them for more days than they were listed.

Let's assume you managed to buy a new stock (listed on IPO) on the first day at the [Adj Close] price]. Your strategy is to hold for exactly X full days (where X is between 1 and 30) and sell at the "Adj. Close" price in X days (e.g., if X=1, you sell on the next day). Find X, when the 75% quantile growth (among 185 investments) is the highest.

HINTs:

    You can generate 30 additional columns: growth_future_1d ... growth_future_30d, join that with the table of min_dates (first day when each stock has data on Yahoo Finance), and perform vector operations on the resulting dataset.
    You can use the DataFrame.describe() function to get mean, min, max, 25-50-75% quantiles.

Additional:

    You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%.
    What's your recommendation: Do you suggest pursuing this strategy for an optimal X?


In [87]:
import pandas as pd
import requests
import yfinance as yf

# Function to retrieve IPO data from stockanalysis.com
def get_ipo_data(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    return pd.read_html(response.text)[0]

# Retrieve IPO data for 2023 and 2024
url_2023 = "https://stockanalysis.com/ipos/2023/"
url_2024 = "https://stockanalysis.com/ipos/2024/"

ipo_2023 = get_ipo_data(url_2023)
ipo_2024 = get_ipo_data(url_2024)

# Combine IPO data
ipo_data = pd.concat([ipo_2023, ipo_2024])

# Filter IPOs before March 1, 2024
ipo_data['IPO Date'] = pd.to_datetime(ipo_data['IPO Date'])
ipo_data = ipo_data[ipo_data['IPO Date'] < '2024-03-01']

# Get tickers (adjusting symbol names if necessary)
tickers = ipo_data['Symbol'].tolist()
tickers.remove('RYZB')  # Remove 'RYZB' as it is no longer available on Yahoo Finance

# Get OHLCV daily prices for all tickers
start_date = '2023-01-01'
end_date = '2024-03-01'
prices = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

# Display the first few rows of the prices DataFrame
print(prices.head())


[*********************100%%**********************]  184 of 184 completed

4 Failed downloads:
['JVSA', 'DYCQ', 'LEGT']: Exception("%ticker%: Data doesn't exist for startDate = 1672549200, endDate = 1709269200")
['PTHR']: Exception('%ticker%: No timezone found, symbol may be delisted')


Ticker      AACT  AESI  AFJK  AHR  AITR  AIXI  ALCY  ANL  ANRO  ANSC  ...  \
Date                                                                  ...   
2023-01-03   NaN   NaN   NaN  NaN   NaN   NaN   NaN  NaN   NaN   NaN  ...   
2023-01-04   NaN   NaN   NaN  NaN   NaN   NaN   NaN  NaN   NaN   NaN  ...   
2023-01-05   NaN   NaN   NaN  NaN   NaN   NaN   NaN  NaN   NaN   NaN  ...   
2023-01-06   NaN   NaN   NaN  NaN   NaN   NaN   NaN  NaN   NaN   NaN  ...   
2023-01-09   NaN   NaN   NaN  NaN   NaN   NaN   NaN  NaN   NaN   NaN  ...   

Ticker      VHAI  VSME  VTMX  WBUY  WLGS  WRNT  YGFGF  YIBO  ZJYL  ZKH  
Date                                                                    
2023-01-03   NaN   NaN   NaN   NaN   NaN   NaN    NaN   NaN   NaN  NaN  
2023-01-04   NaN   NaN   NaN   NaN   NaN   NaN    NaN   NaN   NaN  NaN  
2023-01-05   NaN   NaN   NaN   NaN   NaN   NaN    NaN   NaN   NaN  NaN  
2023-01-06   NaN   NaN   NaN   NaN   NaN   NaN    NaN   NaN   NaN  NaN  
2023-01-09   NaN   NaN

In [123]:
import pandas as pd
import yfinance as yf
from requests import get  # Import get function directly from requests

def get_and_analyze_ipos(start_year, end_year):
  """
  Fetches IPO data for a given year range, retrieves prices, calculates growth rates, 
  and finds the optimal holding period.

  Args:
      start_year (int): Starting year for IPO data retrieval.
      end_year (int): Ending year for IPO data retrieval.

  Returns:
      int: The optimal holding period (number of days) with the highest 75th percentile growth.
  """
  # Fetch and combine IPO data
  ipos = pd.DataFrame()
  for year in range(start_year, end_year + 1):
    url = f"https://stockanalysis.com/ipos/{year}/"
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = get(url, headers=headers)
    ipos = pd.concat([ipos, pd.read_html(response.text)[0]], ignore_index=True)
  ipos['IPO Date'] = pd.to_datetime(ipos['IPO Date'])

  # Filter IPOs and define adjustments (optional)
  filtered_ipo = ipos[(ipos['IPO Date'] < '2024-03-01') & (ipos['Symbol'] != 'RYZB')]
  ticker_adjustments = {"PTHR": "PTHRF", "IBAC": "IBACU"}  # Optional adjustments

  # Download prices and calculate growth rates
  price_data = {}
  for symbol in filtered_ipo['Symbol']:
    adjusted_symbol = ticker_adjustments.get(symbol, symbol)
    try:
      df = yf.download(adjusted_symbol, start="2023-01-01", end="2024-03-01")
      if df is not None:
        for i in range(1, 31):
          df[f'growth_{i}d'] = ((df['Adj Close'].shift(-i) / df['Adj Close'] - 1) * 100).round(2)
        price_data[symbol] = df
    except Exception as e:
      print(f"Error downloading {symbol}: {e}")

 # Combine price data (if any) and calculate growth percentiles
  results = pd.DataFrame()
  for symbol, df in price_data.items():
    results = pd.concat([results, df[[f'growth_{i}d' for i in range(1, 31)]]])
  growth_quantile = results.quantile(0.75)

  # Find optimal holding period with highest 75th percentile growth
  optimal_days = growth_quantile.idxmax()

  # Analyze additional statistics (mean, median)
  growth_describe = results.describe(percentiles=[.5])  # Calculate mean, median

# Run the analysis and print the optimal holding period
optimal_period = get_and_analyze_ipos(2023, 2024)
print(f"The optimal holding period is {optimal_days} days.")


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

The optimal holding period is 28 days.


In [122]:
import pandas as pd
import yfinance as yf
from requests import get  # Import get function directly from requests

def get_and_analyze_ipos(start_year, end_year):
  """
  Fetches IPO data for a given year range, retrieves prices, calculates growth rates, 
  and finds the optimal holding period. Analyzes additional statistics.

  Args:
      start_year (int): Starting year for IPO data retrieval.
      end_year (int): Ending year for IPO data retrieval.

  Returns:
      int: The optimal holding period (number of days) with the highest 75th percentile growth.
  """
  # Fetch and combine IPO data
  ipos = pd.DataFrame()
  for year in range(start_year, end_year + 1):
    url = f"https://stockanalysis.com/ipos/{year}/"
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = get(url, headers=headers)
    ipos = pd.concat([ipos, pd.read_html(response.text)[0]], ignore_index=True)
  ipos['IPO Date'] = pd.to_datetime(ipos['IPO Date'])

  # Filter IPOs and define adjustments (optional)
  filtered_ipo = ipos[(ipos['IPO Date'] < '2024-03-01') & (ipos['Symbol'] != 'RYZB')]
  ticker_adjustments = {"PTHR": "PTHRF", "IBAC": "IBACU"}  # Optional adjustments

  # Download prices and calculate growth rates
  price_data = {}
  for symbol in filtered_ipo['Symbol']:
    adjusted_symbol = ticker_adjustments.get(symbol, symbol)
    try:
      df = yf.download(adjusted_symbol, start="2023-01-01", end="2024-03-01")
      if df is not None:
        for i in range(1, 31):
          df[f'growth_{i}d'] = ((df['Adj Close'].shift(-i) / df['Adj Close'] - 1) * 100).round(2)
        price_data[symbol] = df
    except Exception as e:
      print(f"Error downloading {symbol}: {e}")

  # Combine price data (if any) and calculate growth percentiles
  results = pd.DataFrame()
  for symbol, df in price_data.items():
    results = pd.concat([results, df[[f'growth_{i}d' for i in range(1, 31)]]])
  growth_quantile = results.quantile(0.75)

  # Find optimal holding period with highest 75th percentile growth
  optimal_days = growth_quantile.idxmax()

  # Analyze additional statistics (mean, median)
  growth_describe = results.describe(percentiles=[.5])  # Calculate mean, median

  # Print results
  print(f"The optimal holding period is {optimal_days} days.")
  print(f"Growth statistics (across all investments for each holding period):")
  print(growth_describe)

  # Recommendation (Avoid pursuing for optimal X)
  print("\nRecommendation:")
  print("While the strategy focuses on the top 25% performers (75th percentile growth),")
  print("the overall mean and median growth across all investments are negative for most holding periods.")
  print("This suggests the strategy is risky and may not be profitable in general.")
  print("It's generally not recommended to chase 'lucky' investments based on short-term growth.")

  return optimal_days

# Run the analysis and print results
optimal_period = get_and_analyze_ipos(2023, 2024)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

The optimal holding period is growth_28d days.
Growth statistics (across all investments for each holding period):
          growth_1d     growth_2d     growth_3d     growth_4d     growth_5d  \
count  25773.000000  25592.000000  25411.000000  25231.000000  25052.000000   
mean      -0.019909      0.009616      0.058067      0.096509      0.152900   
std       10.744505     15.177685     18.599278     21.301001     24.182108   
min      -88.990000    -93.550000    -93.790000    -93.630000    -93.550000   
50%        0.000000      0.000000      0.000000      0.000000      0.000000   
max      582.690000    597.130000    662.560000    765.050000    795.260000   

          growth_6d     growth_7d     growth_8d     growth_9d    growth_10d  \
count  24873.000000  24694.000000  24515.000000  24336.000000  24159.000000   
mean       0.223766      0.260309      0.321074      0.401132      0.471337   
std       27.032403     28.971771     30.999573     33.078063     34.979911   
min      -93.70

Question 3: Is Growth Concentrated in the Largest Stocks?

Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?

Reuse [Code Snippet 5] to obtain OHLCV stats for 33 stocks for 10 full years of data (2014-01-01 to 2023-12-31). You'll need to download slightly more data (7 periods before 2014-01-01 to calculate the growth_7d for the first 6 days correctly):

US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

Now let's add the top 12-22 stocks (as of end-April 2024):

NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

You should be able to obtain stats for 33 LARGEST STOCKS and 32 LARGE STOCKS (from the actual stats on Yahoo Finance)

Calculate growth_7d for every stock and every day. Get the average daily growth_7d for the LARGEST_STOCKS group vs. the LARGE_STOCKS group.

For example, for the first of data you should have:
Date 	ticker_category 	growth_7d
2014-01-01 	LARGE 	1.011684
2014-01-01 	LARGEST 	1.011797

On that day, the LARGEST group was growing faster than LARGE one (new stocks).

Calculate the number of days when the LARGE GROUP (new smaller stocks) outperforms the LARGEST GROUP, divide it by the total number of trading days (which should be 2595 days), and convert it to a percentage (closest INTEGER value). For example, if you find that 1700 out of 2595 days meet this condition, it means that 1700/2595 = 0.655, or approximately 66% of days, the LARGE stocks were growing faster than the LARGEST ones. This suggests that you should consider extending your dataset with more stocks to seek higher growth.

HINT: you can use pandas.pivot_table() to "flatten" the table (LARGE and LARGEST growth_7d as columns)

In [31]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta

# Define the lists of tickers
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO', 'V', 'JPM']
EU_STOCKS = ['NVO', 'MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE', 'IDEXY', 'CDI.PA']
INDIA_STOCKS = ['RELIANCE.NS', 'TCS.NS', 'HDB', 'BHARTIARTL.NS', 'IBN', 'SBIN.NS', 'LICI.NS', 'INFY', 'ITC.NS', 'HINDUNILVR.NS', 'LT.NS']
LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

NEW_US = ['TSLA', 'WMT', 'XOM', 'UNH', 'MA', 'PG', 'JNJ', 'MRK', 'HD', 'COST', 'ORCL']
NEW_EU = ['PRX.AS', 'CDI.PA', 'AIR.PA', 'SU.PA', 'ETN', 'SNY', 'BUD', 'DTE.DE', 'ALV.DE', 'MDT', 'AI.PA', 'EL.PA']
NEW_INDIA = ['BAJFINANCE.NS', 'MARUTI.NS', 'HCLTECH.NS', 'TATAMOTORS.NS', 'SUNPHARMA.NS', 'ONGC.NS', 'ADANIENT.NS', 'ADANIENT.NS', 'NTPC.NS', 'KOTAKBANK.NS', 'TITAN.NS']
LARGE_STOCKS = NEW_US + NEW_EU + NEW_INDIA

# Function to fetch OHLCV data for a ticker
def fetch_prices(ticker, start_date, end_date):
    try:
        prices = yf.download(ticker, start=start_date, end=end_date)
        return prices
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

# Fetch OHLCV data for LARGEST_STOCKS and LARGE_STOCKS
start_date = '2013-12-24'  # 7 periods before 2014-01-01
end_date = '2023-12-31'
all_tickers = LARGEST_STOCKS + LARGE_STOCKS

prices = {}
for ticker in all_tickers:
    prices[ticker] = fetch_prices(ticker, start_date, end_date)

# Calculate growth_7d for each stock and each day
for ticker, df in prices.items():
    df['growth_7d'] = df['Adj Close'] / df['Adj Close'].shift(7)

# Flatten the table and calculate average daily growth_7d for LARGEST_STOCKS and LARGE_STOCKS
flattened_prices = pd.concat(prices.values(), keys=prices.keys(), names=['Ticker'])
pivot_table = flattened_prices.pivot_table(index='Date', columns='Ticker', values='growth_7d')

average_growth_largest = pivot_table[LARGEST_STOCKS].mean(axis=1)
average_growth_large = pivot_table[LARGE_STOCKS].mean(axis=1)

# Calculate the number of days when LARGE GROUP outperforms LARGEST GROUP
outperform_days = (average_growth_large > average_growth_largest).sum()

# Calculate the share of outperforming days as a percentage
total_days = len(average_growth_largest)
share_outperform_days = int((outperform_days / total_days) * 100)

print(f"Share of days when LARGE STOCKS outperform LARGEST STOCKS: {share_outperform_days}%")


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Share of days when LARGE STOCKS outperform LARGEST STOCKS: 47%


Question 4: Trying Another Technical Indicators strategy

What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?

First, run the entire Colab to obtain the full DataFrame of data (after [Code Snippet 9]), and truncate it to the last full 10 years of data (2014-01-01 to 2023-12-31). If you encounter any difficulties running the Colab - you can download it using this link.

Let's assume you've learned about the awesome CCI indicator (Commodity Channel Index), and decided to use only it for your operations.

You defined the "defensive" value of a high threshould of 200, and you trade only on Fridays (Date.dt.dayofweek()==4).

That is, every time you see that CCI is >200 for any stock (out of those 33), you'll invest $1000 (each record when CCI>200) at Adj.Close price and hold it for 1 week (5 trading days) in order to sell at the Adj. Close price.

What's the expected gross profit (no fees) that you get in THOUSANDS $ (closest integer value) over many operations in 10 years? One operation calculations: if you invested $1000 and received $1010 in 5 days - you add $10 to gross profit, if you received $980 - add -$20 to gross profit. You need to sum these results over all trades (460 times in 10 years).

Additional:

    Add an approximate fees calculation over the 460 trades from this calculator https://www.degiro.ie/fees/calculator (Product:"Shares, USA and Canada;" Amount per transaction: "1000 EUR"; Transactions per year: "460")
    are you still profitable on those trades?


In [42]:
import pandas as pd

# Specify the path to your Parquet file
file_path = "C:/Users/msofy/Downloads/stock-markets-analytics-zoomcamp-2024/02-dataframe-analysis/stocks_df_combined_trunc_2014_2023.parquet.brotli"

# Load the Parquet file into a DataFrame
data = pd.read_parquet(file_path)
print(data)

             Open         High          Low        Close  Adj Close_x  \
7011    37.349998    37.400002    37.099998    37.160000    31.233059   
7012    37.200001    37.220001    36.599998    36.910000    31.022930   
7013    36.849998    36.889999    36.110001    36.130001    30.367352   
7014    36.330002    36.490002    36.209999    36.410000    30.602673   
7015    36.000000    36.139999    35.580002    35.759998    30.056356   
...           ...          ...          ...          ...          ...   
5338  3424.000000  3496.000000  3408.600098  3477.949951  3477.949951   
5339  3477.949951  3508.350098  3477.949951  3490.050049  3490.050049   
5340  3510.000000  3549.000000  3504.149902  3544.000000  3544.000000   
5341  3545.000000  3559.949951  3500.500000  3518.050049  3518.050049   
5342  3531.000000  3540.000000  3495.000000  3526.000000  3526.000000   

          Volume Ticker  Year      Month  Weekday  ... growth_brent_oil_7d  \
7011  30632200.0   MSFT  2014 2014-01-01     

In [57]:
import pandas as pd

# Load the DataFrame containing the trading data
df = pd.read_parquet("C:/Users/msofy/Downloads/stock-markets-analytics-zoomcamp-2024/02-dataframe-analysis/stocks_df_combined_trunc_2014_2023.parquet.brotli")

# Filter data to include only the last full 10 years
start_date = '2014-01-01'
end_date = '2023-12-31'
data = data[(data['Date'] >= start_date) & (data['Date'] <= end_date)]

# Filter data to include only Fridays
friday_data = data[data['Date'].dt.dayofweek == 4]

# Filter data to include only records where CCI > 200
cci_threshold = 200
cci_data = friday_data[friday_data['cci'] > cci_threshold]

# Initialize gross profit
gross_profit = 0

# Calculate profit or loss for each trade
for index, row in cci_data.iterrows():
    ticker = row['Ticker']
    price_invested = row['Adj Close_x']
    
    # Find the price after 5 trading days
    sell_date = row['Date'] + timedelta(days=5)
    sell_price_data = data[(data['Date'] >= sell_date) & (data['Date'] <= sell_date + timedelta(days=1)) & (data['Ticker'] == ticker)]
    if not sell_price_data.empty:
        price_sold = sell_price_data.iloc[0]['Adj Close_x']
        profit_loss = (price_sold - price_invested) * 1000  # Assuming $1000 invested
        gross_profit += profit_loss

# Convert gross profit to thousands of dollars (rounded to the nearest integer)
gross_profit_thousands = round(gross_profit / 1000)

print("Expected gross profit (in THOUSANDS of $):", gross_profit_thousands)


Expected gross profit (in THOUSANDS of $): 265


[EXPLORATORY] Question 5: Finding Your Strategy for IPOs

You've seen in the first questions that the median and average investments are negative in IPOs, and you can't blindly invest in all deals.

How would you correct/refine the approach? Briefly describe the steps and the data you'll try to get (it should be generally feasible to do it from public sources - no access to internal data of companies)?

E.g. (some ideas) Do you want to focus on the specific vertical? Do you want to build a smart comparison vs. existing stocks on the market? Or you just will want to get some features (which features?) like total number of people in a company to find a segment of "successful" IPOs?